# News Analysis by FinMAS

This notebook can be used to run an analysis by News Analysis crew instead of using the UI / panel app.

In [ ]:
# Convenience function to change the working directory to the root of the project if necessary
import os
from pathlib import Path

if not Path("finmas").exists():
    os.chdir("..")
# print(f"Current working directory: {Path.cwd()}")

In [ ]:
%%capture
import time

from IPython.display import Markdown, display

from finmas.constants import defaults
from finmas.crews import NewsAnalysisCrew
from finmas.crews.utils import CrewRunMetrics, get_yaml_config_as_markdown, save_crew_output
from finmas.utils.common import format_time_spent

To be able to run this notebook make sure the necessary API_KEY for the LLM is set in the `.env` file
or set in the environment variables manually

| provider | API_KEY |
| --- | --- |
| groq | GROQ_API_KEY |
| openai | OPENAI_API_KEY |

## Configuration

List of Groq models: https://console.groq.com/settings/limits

In [ ]:
TICKER = "TSLA"
NEWS_START_DATE = defaults["news_start_date"]
NEWS_END_DATE = defaults["news_end_date"]

# Groq setup
# LLM_PROVIDER = "groq"
# LLM_MODEL = "llama3-8b-8192"
# EMBEDDING_MODEL = defaults["hf_embedding_model"]

# Change the model here if needed
# OpenAI setup
LLM_PROVIDER = "openai"
# LLM_MODEL = "gpt-4o"
LLM_MODEL = "gpt-4o-mini"
EMBEDDING_MODEL = defaults["openai_embedding_model"]

# LLM configuration
LLM_TEMPERATURE = 0.0
LLM_MAX_TOKENS = 1024

SIMILARITY_TOP_K = 3  # Number of similar news articles to send to the LLM

CREW_NAME = "news"

## Crew Configuration

In [ ]:
config_path = Path.cwd() / f"finmas/crews/{CREW_NAME}/config"

### Agents

In [ ]:
display(Markdown(get_yaml_config_as_markdown(config_path, "agents", inputs={"ticker": TICKER})))

### News Analyzer

- **Role**: Financial News Analyst
- **Goal**: Extract and analyze key information from individual news items to provide a deep understanding of events impacting the company TSLA.
- **Backstory**: You are an experienced financial news analyst with a strong focus on identifying key events and interpreting their implications for a company's financial performance and market standing.

### Sentiment Analyzer

- **Role**: Financial Sentiment Expert
- **Goal**: Assess the sentiment of news articles to determine public perception and potential market impacts for TSLA.
- **Backstory**: You are a specialist in sentiment analysis, with deep knowledge of financial markets and an ability to evaluate how media sentiment can influence investor behavior and company reputation.

### News Summarizer

- **Role**: Financial News Summarizer
- **Goal**: Synthesize analyzed data and generate a coherent and insightful summary of news events, market sentiment, and key company updates for TSLA.
- **Backstory**: You are a skilled financial journalist with expertise in summarizing complex financial news into accessible, concise reports that assist stakeholders in making informed decisions.


### Tasks Definition

In [ ]:
display(Markdown(get_yaml_config_as_markdown(config_path, "tasks", inputs={"ticker": TICKER})))

### News Analyzer Task

- **Description**: Analyze recent news articles about the company TSLA and provide a detailed report highlighting the most impactful events and notable news stories. Focus on events that could significantly affect the company's financial standing or public perception.
- **Expected Output**: A comprehensive news analysis report including key events, impactful news stories, and an assessment of their potential implications for the company.
- **Agent**: news_analyzer

### Sentiment Analysis Task

- **Description**: Conduct a sentiment analysis on the news articles related to TSLA, determining the overall tone and public perception of the company. Include insights into whether the sentiment is positive, negative, or neutral, and explain the reasons behind this sentiment.
- **Expected Output**: A detailed sentiment analysis report that provides an overall sentiment score, categorizes each article by tone, and discusses the potential effects of the sentiment on the company's market position.
- **Agent**: sentiment_analyzer

### News Summary Task

- **Description**: Summarize the key insights from the news articles for TSLA, providing an overview of important events, market sentiment, and significant company news. The summary should present a holistic view of the news landscape for the company, highlighting both the qualitative and quantitative aspects.
- **Expected Output**: A clear and concise news summary report that includes key insights, categorized sections for market sentiment, major events, and relevant company news, with enough detail to inform strategic decision-making.
- **Agent**: news_summarizer



## Benzinga News Fetcher

We demonstrate the news fetcher here to get the news from Benzinga via the Alpaca News API.
If the news already have been stored locally, then the local storage will be used.

In [ ]:
import pandas as pd

from finmas.data.news.benzinga_news import BenzingaNewsFetcher
from finmas.utils.common import to_datetime

news_fetcher = BenzingaNewsFetcher()

news_records = news_fetcher.get_news(
    ticker=TICKER,
    start=to_datetime(NEWS_START_DATE),
    end=to_datetime(NEWS_END_DATE),
)

df = pd.DataFrame.from_records(news_records)
df["published"] = pd.to_datetime(df["published"]).dt.strftime("%Y-%m-%d")
display(Markdown(f"Number of news articles: {len(df)}"))
display(
    Markdown(
        f"First publish date: {df['published'].min()} Max publish date: {df['published'].max()}"
    )
)

Number of news articles: 19

First publish date: 2024-11-06 Max publish date: 2024-11-09

In [ ]:
display(df[["title", "published", "author", "num_symbols"]])

,title,published,author,num_symbols
0,As Tesla Market Capitalization Soars Past $1 T...,2024-11-09,Ananya Gairola,1
1,Elon Musk To Ditch Tesla Gigafactory In Mexico...,2024-11-08,Chris Katje,1
2,Trump's Win Sends Wall Street To Record Highs;...,2024-11-08,Piero Cingari,4
3,Tesla's China Sales Plunge to 6-Month Low In O...,2024-11-08,Nabaparna Bhattacharya,1
4,Tesla Opens Cybertruck Lease Option For As Low...,2024-11-08,Anan Ashraf,1
5,Rivian Vs Tesla: How The EV Rivals Stack Up In...,2024-11-08,Kaustubh Bagalkote,3
6,Elon Musk Reveals Toughest Part Of Building Te...,2024-11-08,Anan Ashraf,1
7,"Rivian, Draftkings, Block, Pinterest, And Tesl...",2024-11-08,Benzinga Neuro,5
8,Elon Musk Shines In 'Trump Sweepstakes:' A Big...,2024-11-08,Kaustubh Bagalkote,1
9,3 Stocks Set For Major Upside From Inflation R...,2024-11-07,Stjepan Kalinic,4


## Create Crew

This section creates the crew with the configuration of agents and tasks defined above.
The crew will use the News Records shown above to summarize and comment on the sentiment of the news.

In [ ]:
# %%capture
start = time.time()
crew = NewsAnalysisCrew(
    ticker=TICKER,
    records=news_records,
    llm_provider=LLM_PROVIDER,
    llm_model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    max_tokens=LLM_MAX_TOKENS,
    embedding_model=EMBEDDING_MODEL,
    similarity_top_k=SIMILARITY_TOP_K,
    news_source="Benzinga",
    news_start=NEWS_START_DATE,
    news_end=NEWS_END_DATE,
)

In [ ]:
from finmas.crews.utils import get_index_creation_metrics

index_creation_metrics_message = get_index_creation_metrics(crew)
display(Markdown(index_creation_metrics_message))

Index Creation Metrics:  
Embedding Model: text-embedding-3-small  
Time spent: 1s  
Number of nodes: 19  
Text length: 52788  
Chunk size: 1024 tokens  
Chunk overlap: 200 tokens  
Total embedding token count: 12413  
Estimated embedding model cost for total tokens: $0.00024826  




## Run the analysis

In [ ]:
%%capture
inputs = {"ticker": TICKER}  # "form": filing.form}
output = crew.crew().kickoff(inputs=inputs)
time_spent = time.time() - start
display(f"Time spent: {format_time_spent(time_spent)}")

## Save the results

In [ ]:
crew_run_metrics = CrewRunMetrics(
    config=crew.config, token_usage=output.token_usage, time_spent=time_spent
)
file_path = save_crew_output(crew_run_metrics, output.raw, index_creation_metrics_message)
display(f"Output saved to: {file_path}")

'Output saved to: output/news/TSLA_news_analysis_20241118_151656.md'

## Display the results

In [ ]:
crew_output = file_path.read_text(encoding="utf-8")
display(Markdown(crew_output))

# Crew Output

**Tesla (TSLA) Recent News Summary Report**

**Introduction:**  
This report aims to synthesize recent news articles concerning Tesla, Inc. (TSLA) to identify key events that could significantly impact the company's financial performance and public perception. The analysis focuses on various themes, including financial results, product innovations, regulatory changes, market competition, and strategic partnerships.

**Key Insights Section:**

**Market Sentiment Overview:**  
- **Overall Sentiment Score**: 0.6 (on a scale of -1 to 1)  
- **Overall Tone**: Positive  
The sentiment analysis indicates a predominantly positive outlook for Tesla, driven by a surge in stock prices and market capitalization. However, concerns regarding declining sales in China temper this optimism.

**Major Events Summary:**
1. **Market Capitalization Surge**:  
   Tesla's market capitalization has surpassed $1 trillion, largely due to a rally in stock prices following political developments, particularly CEO Elon Musk's support for President-elect Donald Trump. Analysts believe this could create favorable conditions for Tesla's growth.  
   - Source: [MarketWatch, November 2023]

2. **Stock Performance Post-Election**:  
   Following Trump's election victory, Tesla's stock rose nearly 29% within a week, boosting the company's market value and elevating Musk's personal wealth beyond $300 billion.  
   - Source: [CNBC, November 2023]

3. **Sales Decline in China**:  
   Despite the positive market sentiment, Tesla's sales in China have declined, with October marking the weakest month for sales since April. However, year-to-date sales figures still reflect an increase compared to the previous year.  
   - Source: [Reuters, November 2023]

4. **Analyst Predictions on Autonomous Driving**:  
   Analysts, including Dan Ives, express optimism that Trump's presidency could accelerate Tesla's autonomous driving initiatives, potentially unlocking significant long-term value for the company.  
   - Source: [Bloomberg, November 2023]

**Compendium of Relevant News:**
- **Positive Sentiment Articles**:  
   - "Tesla's stock surges post-election rally": TSLA shares rose over 14% following Trump's victory, with analysts raising price forecasts and maintaining a Buy rating.  
   - "Tesla crosses $1 trillion market cap": The company's market capitalization surpassed $1 trillion, driven by optimism about growth in the autonomous vehicle sector.

- **Negative Sentiment Articles**:  
   - "Tesla's sales in China fall to six-month low": October sales in China dropped to 68,280 vehicles, raising concerns about market performance in a key region.

- **Neutral Sentiment Articles**:  
   - "Tesla introduces leasing options for Cybertruck": Tesla offers leasing for the Cybertruck at $999 per month, presenting factual information without strong opinions.

**Conclusion:**  
In summary, Tesla is currently navigating a landscape of mixed developments. The significant increase in market capitalization and stock performance is encouraging, reflecting strong investor confidence. However, challenges in the Chinese market, evidenced by declining sales, could impact the company's growth trajectory. The potential acceleration of autonomous driving initiatives presents new opportunities but also comes with regulatory scrutiny and competition. Continuous monitoring of these events will be essential for understanding their long-term implications for Tesla, Inc. Stakeholders should remain vigilant regarding the interplay between positive market sentiment and the challenges posed by international sales performance.

## Crew Run Metrics

Total tokens: 9667 Prompt tokens: 6603  
Successful Requests: 7  
Estimated LLM Model cost for total tokens: $0.00282885  

Time spent: 1m 8s

## Inputs

News Source: Benzinga
Date range: 2024-10-15 - 2024-11-10
## Configuration

Crew Name: news  
Ticker: TSLA  
LLM: openai / gpt-4o-mini  
Temperature: 0.0 Max tokens: 1024  
Agent Configuration:  
Max iterations: 10 Max requests per minute: 30  
Embedding Model: text-embedding-3-small similarity_top_k: 3  

## Agents

### News Analyzer

- **Role**: Financial News Analyst
- **Goal**: Extract and analyze key information from individual news items to provide a deep understanding of events impacting the company {ticker}.
- **Backstory**: You are an experienced financial news analyst with a strong focus on identifying key events and interpreting their implications for a company's financial performance and market standing.

### Sentiment Analyzer

- **Role**: Financial Sentiment Expert
- **Goal**: Assess the sentiment of news articles to determine public perception and potential market impacts for {ticker}.
- **Backstory**: You are a specialist in sentiment analysis, with deep knowledge of financial markets and an ability to evaluate how media sentiment can influence investor behavior and company reputation.

### News Summarizer

- **Role**: Financial News Summarizer
- **Goal**: Synthesize analyzed data and generate a coherent and insightful summary of news events, market sentiment, and key company updates for {ticker}.
- **Backstory**: You are a skilled financial journalist with expertise in summarizing complex financial news into accessible, concise reports that assist stakeholders in making informed decisions.
## Tasks

### News Analyzer Task

- **Description**: Analyze recent news articles about the company {ticker} and provide a detailed report highlighting the most impactful events and notable news stories. Focus on events that could significantly affect the company's financial standing or public perception.
- **Expected Output**: A comprehensive news analysis report including key events, impactful news stories, and an assessment of their potential implications for the company.
- **Agent**: news_analyzer

### Sentiment Analysis Task

- **Description**: Conduct a sentiment analysis on the news articles related to {ticker}, determining the overall tone and public perception of the company. Include insights into whether the sentiment is positive, negative, or neutral, and explain the reasons behind this sentiment.
- **Expected Output**: A detailed sentiment analysis report that provides an overall sentiment score, categorizes each article by tone, and discusses the potential effects of the sentiment on the company's market position.
- **Agent**: sentiment_analyzer

### News Summary Task

- **Description**: Summarize the key insights from the news articles for {ticker}, providing an overview of important events, market sentiment, and significant company news. The summary should present a holistic view of the news landscape for the company, highlighting both the qualitative and quantitative aspects.
- **Expected Output**: A clear and concise news summary report that includes key insights, categorized sections for market sentiment, major events, and relevant company news, with enough detail to inform strategic decision-making.
- **Agent**: news_summarizer



Index Creation Metrics:  
Embedding Model: text-embedding-3-small  
Time spent: 1s  
Number of nodes: 19  
Text length: 52788  
Chunk size: 1024 tokens  
Chunk overlap: 200 tokens  
Total embedding token count: 12413  
Estimated embedding model cost for total tokens: $0.00024826  


